In [1]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
from hyrax import Hyrax

toml_path = "./baselinecls_testing_runtime_config.toml"
h = Hyrax(config_file=toml_path)
h.set_config("data_set.PhotoEventsDataset.use_oversampling", True)
dataset = h.prepare()

/Users/drew/opt/miniconda3/envs/applecider/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
Cannot find default_config.toml for torch.optim.Adam.
Runtime config contains key or section 'HyraxBaselineCLS' which has no default defined. All configuration keys and sections must be defined in /Users/drew/code/hyrax/src/hyrax/hyrax_default_config.toml
Runtime config contains key or section 'lr' which has no default defined. All configuration keys and sections must be defined in /Users/drew/code/hyrax/src/hyrax/hyrax_default_config.toml
Runtime config contains key or section 'train' which has no default defined. All configuration keys and sections must be defined in /Users/drew/code/hyrax/src/hyrax/hyrax_default_config.toml
Runtime config c

In [ ]:
import pandas as pd
manifest_df = pd.read_csv("/Users/dbranton/lincc/incubators/photo_events/manifest_train.csv")

taxonomy_mapper = {0: 0,  # SN Ia -> SNI
                                1: 0,  # SN Ib -> SNI
                                2: 0,  # SN Ic -> SNI
                                3: 1,  # SN II -> SNII
                                4: 1,  # SN IIP -> SNII
                                5: 1,  # SN IIn -> SNII
                                6: 1,  # SN IIb -> SNII
                                7: 2,  # Cataclysmic -> CV
                                8: 3,  # AGN -> AGN
                                9: 4,  # Tidal Disruption Event -> TDE
                                }
manifest_df.sort_values("obj_id", inplace=True)

for i in range(10):
    if dataset['train'][i]['data']['label'] != taxonomy_mapper[manifest_df.iloc[i].label]:
        print(f"{dataset['train'][i]['object_id']}, {dataset['train'][i]['data']['label']}, {taxonomy_mapper[manifest_df.iloc[i].label]}")

In [ ]:
manifest_df.query("obj_id == 'ZTF19aavnwzv'").iloc[0].label

In [ ]:
count = 0
for i in range(len(dataset["train"])):
    if dataset["train"][i]["data"]["label"] == 4:
        count += 1
print(count/len(dataset["train"]))
print(count)

In [ ]:
dataset["train"].prepped_datasets["data"].additional_samples_per_class


In [ ]:
len(dataset["train"])

In [ ]:
dataset["train"].sample_data()

In [ ]:
dataset["train"][0]["data"]["photometry"][:, 4:7][20:40]

In [ ]:
dataset["train"][0]

In [ ]:
dataset["infer"][0]

In [ ]:
dataset["infer"].prepped_datasets["data"].manifest_df

In [ ]:
h.config['train']['epochs'] = 1
h.train()

In [ ]:
i_ds = h.infer()

In [ ]:
# Get the predicted classes for each of the inference results
#! Note that we need to apply softmax to the output tensors to get probabilities
import torch
import torch.nn.functional as F
import numpy as np
res_list = []

for i in range(32):
    a = np.load(f"/home/drew/code/applecider/docs/pre_executed/testing/results/20251124-150917-infer-LuHy/batch_{i}.npy")

    for i in a:
        max_ind = np.argmax(F.softmax(torch.as_tensor(i['tensor']), dim=0).numpy())
        res_list.append((str(i['id']), int(max_ind)))

In [ ]:
# Get the known classes for each of the inference results


import pandas as pd
manifest_df = pd.read_csv("../../../data/photo_events/manifest_test.csv")

taxonomy_mapper = {0: 0,  # SN Ia -> SNI
                                1: 0,  # SN Ib -> SNI
                                2: 0,  # SN Ic -> SNI
                                3: 1,  # SN II -> SNII
                                4: 1,  # SN IIP -> SNII
                                5: 1,  # SN IIn -> SNII
                                6: 1,  # SN IIb -> SNII
                                7: 2,  # Cataclysmic -> CV
                                8: 3,  # AGN -> AGN
                                9: 4,  # Tidal Disruption Event -> TDE
                                }

pred_class = []
real_class = []
for id, pred in res_list:
    pred_class.append(pred)
    row = manifest_df[manifest_df["obj_id"] == id]
    real_class.append(taxonomy_mapper[row.iloc[0].label])


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
ConfusionMatrixDisplay(confusion_matrix(real_class, pred_class)).plot()

In [2]:
h.to_onnx()

[2025-12-15 15:29:56,529 hyrax.config_utils:WARNING] Cannot find default_config.toml for torch.nn.CrossEntropyLoss.
[2025-12-15 15:29:56,529 hyrax.config_utils:WARNING] Cannot find default_config.toml for torch.optim.Adam.
[2025-12-15 15:29:56,530 hyrax.config_utils:WARNING] Cannot find default_config.toml for umap.UMAP.
[2025-12-15 15:29:56,531 hyrax.config_utils:INFO] Merging external default config from /Users/drew/code/applecider/src/applecider/default_config.toml
[2025-12-15 15:29:56,547 hyrax.config_utils:WARNING] Runtime config contains key or section 'HyraxBaselineCLS' which has no default defined. All configuration keys and sections must be defined in /Users/drew/code/hyrax/src/hyrax/hyrax_default_config.toml
[2025-12-15 15:29:56,548 hyrax.config_utils:WARNING] Runtime config contains key or section 'lr' which has no default defined. All configuration keys and sections must be defined in /Users/drew/code/hyrax/src/hyrax/hyrax_default_config.toml
[2025-12-15 15:29:56,549 hyrax.

In [4]:
h.engine()

  0%|          | 0/32 [00:00<?, ?it/s]


InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: input_1 for the following indices
 index: 1 Got: 139 Expected: 164
 Please fix either the inputs/outputs or the model.